In [1]:
import pandas as pd

In [2]:
captions_path = './captions.txt'
captions = pd.read_csv(captions_path, sep='\t', header=None)
captions.columns = ['index','captions']
captions_df = pd.DataFrame(captions)
captions_df=captions_df.head(1000)
captions_df.head(10)

,index,captions
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...
5,1001773457_577c3a7d70.jpg#0,A black dog and a spotted dog are fighting
6,1001773457_577c3a7d70.jpg#1,A black dog and a tri-colored dog playing with...
7,1001773457_577c3a7d70.jpg#2,A black dog and a white dog with brown spots a...
8,1001773457_577c3a7d70.jpg#3,Two dogs of different breeds looking at each o...
9,1001773457_577c3a7d70.jpg#4,Two dogs on pavement moving toward each other .


In [3]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from parsel import Selector
import urllib
import urllib.request
import numpy as np
import pandas as pd
import math

In [4]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/Users/prekshyabasnet/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [6]:
#url = "www.google.com"
driver.get("https://www.google.com")
sleep(1)

In [7]:
def save_TopFour(src,j,i):
    urllib.request.urlretrieve(src,'./P_images/%s_%s.jpeg'%(j,i))
    image = {
        "file_name":'%s_%s.jpeg'%(j,i),
        "id":'%s_%s'%(j,i)
    }
    annotation = {
        "caption": val,
        "id": '%s_%s'%(j,i),
        "image_id": '%s_%s.jpeg'%(j,i)
    }
    return image, annotation
            

In [ ]:
from tqdm import tqdm 

images = []
annotations = []

for (j,val) in tqdm(enumerate(captions_df['captions']),desc="Downloading..."):
    sleep(2)
    driver.find_element_by_name('q').send_keys(val)
    driver.find_element_by_xpath("/html/body/div[2]/div[2]/form/div[2]/div[1]/div[3]/center/input[1]").click()
    sleep(2)
    driver.find_element_by_class_name("q.qs").click()
    imgs = driver.find_elements_by_class_name("rg_i.Q4LuWd")

    for i in range(len(imgs)):
        if(i==4):
            break
        else:
            src  = imgs[i].get_attribute('src')
            alt_txt = imgs[i].get_attribute('alt')
            print('./P_images/%s_%s.jpeg'%(j,i))
            image,annotation = save_TopFour(src,j,i)
            images.append(image)
            annotations.append(annotation)
    driver.get("https://www.google.com") 

In [9]:
len(annotations)

1572

In [10]:
len(images)==len(annotations)

True

In [11]:
json_final_200 = {
    "images": images,
    "annotations":annotations
}

In [12]:
import json
with open('./topN_images.json', 'w') as json_file:
    json.dump(json_final_200, json_file)

In [ ]:
import json
# opening the saved json file to preprocess it
with open('topN_images.json') as json_file:
    data = json.load(json_file)

In [11]:
import glob
import re

imgs_file = glob.glob('./selected_pics/*.jpeg')
selected_images = [re.findall("./selected_pics/(.*.jpeg)", file)[0] for file in imgs_file]

In [12]:
len(selected_images)

50

In [17]:
annotations = data['annotations']
annotations[0]

{'caption': 'A child in a pink dress is climbing up a set of stairs in an entry way .',
 'id': '0_0',
 'image_id': '0_0.jpeg'}

In [27]:
images = data['images']
images[0]

{'file_name': '0_0.jpeg', 'id': '0_0'}

In [24]:
list_annotation = []
for image_name in selected_images:
    list_annotation.append([annotate for annotate in annotations if annotate['image_id']==image_name][0])

In [28]:
list_images = []
for image_name in selected_images:
    list_images.append([image for image in images if image['file_name']==image_name][0])

In [29]:
len(list_annotation),len(list_images)

(50, 50)

In [30]:
list_annotation[0],list_images[0]

({'caption': 'A man with two dogs on a beach',
  'id': '288_2',
  'image_id': '288_2.jpeg'},
 {'file_name': '288_2.jpeg', 'id': '288_2'})

In [32]:
nest = {
        "images":list_images,
        "annotations":list_annotation
    }

In [33]:
# Saving the file
with open('./captions_topN2020.json', 'w') as json_file:
    json.dump(nest, json_file)